In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
import matplotlib.pyplot as plt
import glob
from sklearn import svm
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
path_train='../input/a2-data/data/asl_alphabet_train'
train = list(glob.glob(path_train+'/*/*.jpg'))
#train.extend(list(glob.glob(path_train+'/*/nothing.jpg')))
def creatClf(train,color,binary,clf):
    labels = list(map(lambda x:os.path.split(os.path.split(x)[0])[1], train))
    numeric_labels=[]
    for l in labels:
        if l =='del':
            numeric_labels.append(26)
        elif l=='nothing':
            numeric_labels.append(27)
        elif l=='space':
            numeric_labels.append(28)
        else:
            numeric_labels.append(ord(l)-ord('A'))
    train = pd.Series(train, name='FilePath').astype(str)
    labels = pd.Series(labels, name='Label')
    data = pd.concat([train, labels], axis=1)
    data = data.sample(frac=1).reset_index(drop=True)
    Xtrain=[]
    Ytrain=[]
    Xtest=[]
    Ytest=[]
    
    print(len(train))
    for t in range(int(len(train)*.8)) :
        img=cv2.imread(data['FilePath'][t])
        img = cv2.cvtColor(img,color)
        img=cv2.resize(img,(100,100))
        img= cv2.GaussianBlur(img, (3, 3), 0)
        img=cv2.Canny(image=img, threshold1=3, threshold2=100)
        if binary: _,img=cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV)
        Xtrain.append(img)
        Ytrain.append(data['Label'][t])
    for t in range(int(len(train)*.8),int(len(train))):
        img=cv2.imread(data['FilePath'][t])
        img = cv2.cvtColor(img,color)
        img=cv2.resize(img,(100,100))
        img= cv2.GaussianBlur(img, (3, 3), 0)
        img=cv2.Canny(image=img, threshold1=3, threshold2=100)
        if binary: _,img=cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV)
        Xtest.append(img)
        Ytest.append(data['Label'][t])
    lenofimagetrain=len(Xtrain)
    lenofimagetest=len(Xtest)
    flatXtrain= np.array(Xtrain).reshape(lenofimagetrain,-1)
    flatXtest= np.array(Xtest).reshape(lenofimagetest,-1)
    clf.fit(flatXtrain,Ytrain)
    Ypredict=clf.predict(flatXtest)
    #n=0
    #for y in range(len(Ypredict)):
    #    if Ypredict[y]==Ytest[y] : n+=1
    #print(n/200.0)
    print(accuracy_score(Ytest,Ypredict))
    print(precision_score(Ytest,Ypredict,average=None))
    print(recall_score(Ytest,Ypredict,average=None))

#creatClf(train,cv2.COLOR_BGR2RGB,False,svm.SVC())
#creatClf(train,cv2.COLOR_BGR2GRAY,False,svm.SVC())
#creatClf(train,cv2.COLOR_BGR2GRAY,True,svm.SVC()) 
#creatClf(train,cv2.COLOR_BGR2GRAY,True,DecisionTreeClassifier())
#creatClf(train,cv2.COLOR_BGR2RGB,False,DecisionTreeClassifier())
#creatClf(train,cv2.COLOR_BGR2GRAY,False,DecisionTreeClassifier())
#creatClf(train,cv2.COLOR_BGR2RGB,False,KNeighborsClassifier(n_neighbors=3))
#creatClf(train,cv2.COLOR_BGR2GRAY,True,KNeighborsClassifier(n_neighbors=3))
creatClf(train,cv2.COLOR_BGR2GRAY,False,KNeighborsClassifier(n_neighbors=3))
